In [2]:
from genetic_algorithm import *
import re

## hexadecimal generator

In [3]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 2
nr_of_genes = 16
nr_individuals = 4

res = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## Sumuj duble

In [4]:
dic = {}
for nr in res:
    dic[nr] = {}
    for i in res[nr]:
        total = dic.get(i.differ_neuron, 0) + i.weight 
        dic[nr][i.differ_neuron] = total

## grupuj 

In [5]:
dic

{0: {'1mid0mid': -1.2304593316038455,
  '0mid2out': -0.04578055852281398,
  '1in1out': -2.3072180680604304,
  '0in0mid': -1.2122691896841142,
  '0in2out': 1.574851213184801,
  '0in0out': -2.7428048222188313,
  '0mid0mid': -1.1343811994506332,
  '2in0mid': -3.2233175644742866,
  '0mid1out': 0.8535937738440409,
  '0in1out': 1.13633450328094},
 1: {'0mid1out': 3.5564779490309784,
  '0mid0out': 2.7408515183885243,
  '1mid1mid': -2.4274683351136885,
  '1mid0out': 0.5907523271783915,
  '0in0out': 2.4424843583091715,
  '1mid0mid': -3.7046848771555014,
  '1in1out': -2.499252250877461,
  '0in0mid': -1.2360750801159774,
  '0in2out': 3.6448649473523576,
  '0mid1mid': -3.0544788646421486,
  '0mid0mid': 0.6178544178238975,
  '1in1mid': 2.617305051121624},
 2: {'1in0mid': -1.481092629330078,
  '1mid1mid': 1.6876545093850146,
  '1mid1out': 2.7360903403021517,
  '0in0out': -0.7136883869983214,
  '2in0mid': 2.653807416450481,
  '0in2out': -3.1995116740424234,
  '2in1out': 3.061315428048222,
  '0in1mid'

In [13]:
# wybierz wszystkie in_mid

nr_neuronu = 3
print(dic[nr_neuronu])
list_of_keys = []
for i in dic[nr_neuronu]:
    if 'in' in i and 'mid' in i:
        print('in_mid', i)
        list_of_keys.append(i)
        
for i in list_of_keys:
    dic[nr_neuronu].pop(i)

print(dic[nr_neuronu])

{'0mid1out': -2.8990691286433696, '0in2out': -2.022279871814436, '0in0out': -1.7821455821761025, '0mid0mid': -2.805066381809858, '0mid0out': -2.48020753853197}
{'0mid1out': -2.8990691286433696, '0in2out': -2.022279871814436, '0in0out': -1.7821455821761025, '0mid0mid': -2.805066381809858, '0mid0out': -2.48020753853197}


In [14]:
dic[nr_neuronu]

{'0mid1out': -2.8990691286433696,
 '0in2out': -2.022279871814436,
 '0in0out': -1.7821455821761025,
 '0mid0mid': -2.805066381809858,
 '0mid0out': -2.48020753853197}

In [10]:
list_of_out = ['0mid']

d = {}
for n in list_of_out:
    d[n] = [k for k in list_of_keys if n in k]
    
d

{'0mid': ['2in0mid', '0in0mid']}